In [2]:
from __future__ import print_function
from __future__ import absolute_import

In [3]:
import sys
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# You'll generate plots of attention in order to see which parts of an image
# our model focuses on during captioning
import matplotlib.pyplot as plt

/home/sracela/miniconda3/envs/p2c/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sracela/miniconda3/envs/p2c/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sracela/miniconda3/envs/p2c/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sracela/miniconda3/envs/p2c/lib/python3.6/s

InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory

In [4]:
from classes.dataset.Generator import *
from classes.model.pix2code_attention import *

Using TensorFlow backend.


### Images and captions

Transformed images in training dataset to numpy arrays already provided in 

../datasets/web/training_features

Normalized pixel values and resized pictures 

In [5]:
# Store captions and image names in vectors
# Preprocess the images && Preprocess and tokenize the captions

input_path = '../datasets/android/training_features'
output_path = '../bin'

In [6]:
# Load caption annotation files and image files in Dataset
dataset = Dataset()
dataset.load(input_path, generate_binary_sequences=True)
dataset.save_metadata(output_path)
dataset.voc.save(output_path)

Loading data...
Generating sparse vectors...
Dataset size: 86150
Vocabulary size: 20
Input shape: (256, 256, 3)
Output size: 20


In [7]:
len(dataset.partial_sequences[5])


48

In [8]:
# USANDO GENERATOR

gui_paths, img_paths = Dataset.load_paths_only(input_path)

Parsing data...


In [9]:
input_shape = dataset.input_shape
output_size = dataset.output_size
steps_per_epoch = dataset.size / BATCH_SIZE

In [10]:
voc = Vocabulary()
voc.retrieve(output_path)

{'<START>': 0, '<END>': 1, ' ': 2, 'stack': 3, '{': 4, '\n': 5, 'row': 6, 'label': 7, ',': 8, 'btn': 9, '}': 10, 'slider': 11, 'footer': 12, 'btn-notifications': 13, 'switch': 14, 'btn-dashboard': 15, 'btn-search': 16, 'radio': 17, 'check': 18, 'btn-home': 19}


In [11]:

generator = Generator.data_generator(voc, gui_paths, img_paths, batch_size=BATCH_SIZE, generate_binary_sequences=True)

In [ ]:
CONTEXT_LENGTH, BATCH_SIZE

# Model

In [12]:
import tensorflow as tf
# import the necessary packages
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import concatenate

In [13]:
def image_model_sequential(input_shape):
    # initialize the model along with the input shape to be
    # "channels last" ordering
    model = Sequential()
    # define CONV => RELU => MAXPOOL => DROPOUT block
    model.add(Conv2D(32, (3, 3), padding='valid', activation='relu', input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), padding='valid', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='valid', activation='relu'))
    model.add(Conv2D(64, (3, 3), padding='valid', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3, 3), padding='valid', activation='relu'))
    model.add(Conv2D(128, (3, 3), padding='valid', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # classifier
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.3))

    # repeat the structure CONTEXT_LENGHT times (one image per context vector) 
    # repeat_vector_1 = (None,48,1024)
    model.add(RepeatVector(CONTEXT_LENGTH))

    # return the constructed network architecture
    return model

In [ ]:
image_model.summary()

In [ ]:
image_model = image_model_sequential(input_shape) 

In [14]:
from tensorflow.keras.layers import LSTM

In [15]:
def language_model_sequential(output_size):
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=(CONTEXT_LENGTH, output_size)))
    model.add(LSTM(128, return_sequences=True))
    return model

In [ ]:
language_model = language_model_sequential(output_size) 

In [ ]:
language_model.summary()

In [ ]:
class RNN_Decoder(tf.keras.Model):
    def __init__(self, output_size):
        super(RNN_Decoder, self).__init__()
        
        self.lstm1 = LSTM(512, return_sequences=True)
        self.lstm2 = LSTM(512, return_sequences=False)
#         self.fc1 = tf.keras.layers.Dense(self.units)
        self.fc2 = Dense(output_size, activation='softmax')
    def call(self, x):
        x = self.lstm1(x)
        x = self.lstm2(x)
#         x = self.fc1(output)
#         x = tf.reshape(x, (-1, x.shape[2]))
        x = self.fc2(x)
        return x

In [ ]:
def decoder_sequential():
    model = Sequential()
    model.add(LSTM(512, return_sequences=True))
    model.add(LSTM(512, return_sequences=False))
    model.add(Dense(output_size, activation='softmax'))
    return model

In [ ]:
class Pix2Code(Model):
    def __init__(self, input_shape, output_size):
        # call the parent constructor
        super(Pix2Code, self).__init__()
        # define inputs
#         visual_input = Input(shape=init_input_shape)        
#         textual_input = Input(shape=(CONTEXT_LENGTH, output_size))
        
        # define the two feature extractor models
        self.image_model = image_model_sequential(input_shape)
        self.language_model = language_model_sequential(output_size)        

        # define decoder
        self.decoder = decoder_sequential()  
    def call(self, inputs):
        encoded_image = self.image_model(inputs[0])
        encoded_text = self.language_model(inputs[1])
        x = concatenate([encoded_image, encoded_text])
        x = self.decoder(x)
        return x         
        

In [16]:
def pix2code(input_shape, output_size):
    # define inputs
    visual_input = Input(shape=input_shape)        
    textual_input = Input(shape=(CONTEXT_LENGTH, output_size))
    
    
    image_model = image_model_sequential(input_shape)
    language_model = language_model_sequential(output_size)
    
#     decoder = decoder_sequential()
    
    encoded_image = image_model(visual_input)
    encoded_text = language_model(textual_input)            
#     x = concatenate([encoded_image, encoded_text])
#     x = decoder(x)
    decoder = concatenate([encoded_image, encoded_text])

    decoder = LSTM(512, return_sequences=True)(decoder)
    decoder = LSTM(512, return_sequences=False)(decoder)
    decoder = Dense(output_size, activation='softmax')(decoder)
    model = Model(inputs=[visual_input, textual_input], outputs=decoder)
    return model
        

In [17]:
# instantiate a Keras Model sub-class model
model = pix2code(input_shape, output_size)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [18]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint

In [19]:
# initialize the optimizer and compile the model
optimizer = RMSprop(lr=0.0001, clipvalue=1.0)
print("[INFO] training network...")
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

[INFO] training network...


In [20]:
tensorboard = TensorBoard(
    log_dir= "{}/logs".format(output_path),
    histogram_freq=0,
    write_images=True
    )
keras_callbacks = [
    tensorboard,
    EarlyStopping(monitor='loss', patience=5, mode='min', min_delta=0.0001),
    ModelCheckpoint("{}/checkpoints/jupyternotebook_test.h5".format(output_path), monitor='loss', save_best_only=True, mode='min')
    ]

In [22]:
# train the network
seq_Model = model.fit_generator(generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=EPOCHS,
                    verbose=1,
                    callbacks=keras_callbacks)

Epoch 1/3
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
3590/3589 [==============================] - 2102s 585ms/step - loss: 1.6549 - acc: 0.4614
Epoch 2/3
3590/3589 [==============================] - 2048s 570ms/step - loss: 0.4966 - acc: 0.7870
Epoch 3/3
3590/3589 [==============================] - 2027s 565ms/step - loss: 0.3247 - acc: 0.8424


In [23]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 48, 20)]     0                                            
__________________________________________________________________________________________________
sequential_2 (Sequential)       (None, 48, 1024)     104098080   input_3[0][0]                    
__________________________________________________________________________________________________
sequential_3 (Sequential)       (None, 48, 128)      207872      input_4[0][0]                    
______________________________________________________________________________________________

In [24]:
model.save("{}/{}".format(output_path, "jupyterp2c"))

# Evaluate the model

In [25]:
from classes.Sampler import *

In [ ]:
output_path

In [ ]:
loaded_model = Pix2Code(input_shape, output_size)
loaded_model.load("{}/{}".format(output_path, "jupyterp2c"))

In [26]:
sampler = Sampler(output_path, input_shape, output_size, CONTEXT_LENGTH)

{'<START>': 0, '<END>': 1, ' ': 2, 'stack': 3, '{': 4, '\n': 5, 'row': 6, 'label': 7, ',': 8, 'btn': 9, '}': 10, 'slider': 11, 'footer': 12, 'btn-notifications': 13, 'switch': 14, 'btn-dashboard': 15, 'btn-search': 16, 'radio': 17, 'check': 18, 'btn-home': 19}
Vocabulary size: 20
Input shape: (256, 256, 3)
Output size: 20


In [27]:
eval_path = '../datasets/android/eval_set/images/'
beam_width = 3

In [ ]:
a = np.array(loaded_model.get_weights()) - np.array(model.get_weights())

In [ ]:
for i in a:
    print(i.shape)

In [ ]:
np.array(loaded_model.get_weights()[0]).shape ,np.array(model.get_weights()[0]).shape

In [ ]:
equal_len = len(loaded_model.get_weights()) == len(model.get_weights())
# equal_els = sorted(loaded_model.get_weights()) == sorted(model.get_weights())
equal_els = True
assert equal_len and equal_els

In [1]:
for f in os.listdir(eval_path):
    if f.find(".png") != -1:
        evaluation_img = Utils.get_preprocessed_img("{}/{}".format(eval_path, f), IMAGE_SIZE)
        file_name = f[:f.find(".png")]
        print(file_name)
        print("Search with beam width: {}".format(beam_width))
        result, _ = sampler.predict_beam_search(model, np.array([evaluation_img]), beam_width=beam_width)
        print("Result beam: {}".format(result))
        with open("{}/jupyter/{}.gui".format(output_path, file_name), 'w') as out_f:
            out_f.write(result.replace(START_TOKEN, "").replace(END_TOKEN, ""))

NameError: name 'os' is not defined